In [118]:
import numpy as np
import keras
from keras.layers import Conv2D, BatchNormalization,Activation,ZeroPadding2D,MaxPooling2D,Add,AveragePooling2D,Flatten,Dense
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.engine.input_layer import Input
from keras.initializers import glorot_uniform,glorot_normal
from keras.optimizers import Adadelta

import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [101]:
def identity_Block(X,f,filters,stage,block):
    
    #define the name basis
    conv_name_base = "res" + str(stage)+block + "_branch"
    #batch normailization name basis
    bn_name_base = "bn" + str(stage)+block+ "_branch"
    
    #get the filters
    f1,f2,f3 = filters
    #get the shorcut pass
    x_shortcut = X
    
    #create the main path which will hold the conv layers
    
    #First layer main path
    X = Conv2D(f1,kernel_size = (1,1),strides = (1,1),padding='valid',name = conv_name_base + '2a'
               ,kernel_initializer=glorot_normal(seed=0))(X)
    X = BatchNormalization(axis=3,name = bn_name_base+'2a')(X)
    X = Activation('relu')(X)
    
    #second layer main path
    X = Conv2D(f2,kernel_size = (f,f),strides=(1,1),padding='same',name = conv_name_base + '2b'
               ,kernel_initializer = glorot_normal(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
               
    
    #Thrid layer in main path
    X = Conv2D(f3,kernel_size = (1,1),strides = (1,1),padding= 'valid',name = conv_name_base +'2c',
              kernel_initializer = glorot_normal(seed = 0))(X)               
    X = BatchNormalization(axis = 3,name = bn_name_base + '2c')(X)
    #adding the shortcut path
    X = Add()([X ,x_shortcut])
    X = Activation('relu')(X)
               

    return X

In [102]:
def convolutional_block(X,f,filters,stage,block,s = 2):

    #define the name basis
    conv_name_base = "res" + str(stage)+block + "_branch"
    #batch normailization name basis
    bn_name_base = "bn" + str(stage)+block+ "_branch"
    #get the filters
    f1,f2,f3 = filters
    #save the shortcut value
    x_shortcut = X
    #creat the main path
    #create the first layer
    X = Conv2D(f1,kernel_size = (1,1),strides = (s,s),name = conv_name_base +'2a',padding= 'valid'
               ,kernel_initializer=glorot_normal(seed = 0))(X)
    X = BatchNormalization(axis = 3,name = bn_name_base +'2a')(X)
    X = Activation('relu')(X)
    #create the second layer
    X = Conv2D(f2,kernel_size = (f,f),strides = (1,1),name = conv_name_base +'2b',padding= 'same'
               ,kernel_initializer=glorot_normal(seed = 0))(X)
    X = BatchNormalization(axis = 3,name = bn_name_base +'2b')(X)
    X = Activation('relu')(X)
    #create the Third layer
    X = Conv2D(f3,kernel_size = (1,1),strides = (1,1),name = conv_name_base +'2c',padding= 'valid'
               ,kernel_initializer=glorot_normal(seed = 0))(X)
    X = BatchNormalization(axis = 3,name = bn_name_base +'2c')(X)
    X = Activation('relu')(X)
    
    #create the shortcut path
    #the filter size is the same as the last one so they could match
    x_shortcut = Conv2D(f3,kernel_size = (1,1),strides = (s,s),name = conv_name_base +'1',padding= 'valid'
               ,kernel_initializer=glorot_normal(seed = 0))(x_shortcut)
    x_shortcut = BatchNormalization(axis = 3,name = bn_name_base +'1')(x_shortcut)
    
    X = Add()([X,x_shortcut])
    X = Activation('relu')(X)
    
    return X

In [103]:
#declar basic configuration
batch_size =128
img_rows, img_columns =28,28
num_classes =10
(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0],img_rows,img_columns,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_columns,1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255
x_test =x_test/255

y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

input_shape = (img_rows,img_columns,1)

In [115]:
#define the input
model_input = Input(input_shape)


model = ZeroPadding2D(padding = (3,3))(model_input)

#stage 1
model = Conv2D(filters = 64,kernel_size = (7,7),strides= (2,2),name='conv1'
               ,kernel_initializer=glorot_uniform(seed=0))(model)
model = BatchNormalization(axis = 3,name= 'bn_conv1')(model)
model = Activation('relu')(model)
model = MaxPooling2D((3,3),strides=(2,2))(model)

#stage 2
model = convolutional_block(X = model,f = 3, filters =[64,64,256],stage = 2,block = 'a',s = 1)
model = identity_Block(X = model,f = 3,filters= [64,64,256],stage = 2,block = 'b')
model = identity_Block(X = model,f = 3,filters= [64,64,256],stage = 2,block = 'c')

#stage 3
model = convolutional_block(X = model,f = 3, filters =[128,128,512],stage = 3,block = 'a',s = 2)
model = identity_Block(X = model,f = 3,filters= [128,128,512],stage = 3,block = 'b')
model = identity_Block(X = model,f = 3,filters= [128,128,512],stage = 3,block = 'c')
model = identity_Block(X = model,f = 3,filters= [128,128,512],stage = 3,block = 'd')

#stage 4
model = convolutional_block(X = model,f = 3, filters =[256,256,1024],stage = 4,block = 'a',s = 2)
model = identity_Block(X = model,f = 3,filters= [256,256,1024],stage = 4,block = 'b')
model = identity_Block(X = model,f = 3,filters= [256,256,1024],stage = 4,block = 'c')
model = identity_Block(X = model,f = 3,filters= [256,256,1024],stage = 4,block = 'd')
model = identity_Block(X = model,f = 3,filters= [256,256,1024],stage = 4,block = 'e')
model = identity_Block(X = model,f = 3,filters= [256,256,1024],stage = 4,block = 'f')

#stage 5
model = convolutional_block(X = model,f = 3, filters =[512,512,2048],stage = 5,block = 'a',s = 1)
model = identity_Block(X = model,f = 3,filters= [512,512,2048],stage = 5,block = 'b')
model = identity_Block(X = model,f = 3,filters= [512,512,2048],stage = 5,block = 'c')

model = AveragePooling2D((2,2),name = 'avg_pool')(model)
model = Flatten()(model)
model = Dense(num_classes,activation='softmax',name = 'fc' + str(num_classes)
              ,kernel_initializer = glorot_uniform(seed = 0))(model)

model_object = Model(inputs = model_input,outputs = model,name ='ResNet50')

In [119]:
#setup the model for training
model_object.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              #We can specify more metrics that we want to monitor
              metrics=['accuracy'])



In [125]:
#train the model
model_object.fit(x = x_train,y = y_train,
          batch_size = batch_size,
          epochs = 15,
        #this will show is a progress bar put it to 0 so it can remain silent
          verbose =1,
          validation_data = (x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 2420s 40ms/step - loss: 0.1377 - acc: 0.9618 - val_loss: 0.1353 - val_acc: 0.9650
Epoch 2/15
60000/60000 [==============================] - 2393s 40ms/step - loss: 0.2465 - acc: 0.9581 - val_loss: 0.0668 - val_acc: 0.9832
Epoch 3/15
60000/60000 [==============================] - 2392s 40ms/step - loss: 0.1632 - acc: 0.9729 - val_loss: 0.4145 - val_acc: 0.9445
Epoch 4/15
60000/60000 [==============================] - 2392s 40ms/step - loss: 0.0830 - acc: 0.9804 - val_loss: 0.0649 - val_acc: 0.9821
Epoch 5/15
60000/60000 [==============================] - 2401s 40ms/step - loss: 0.0548 - acc: 0.9870 - val_loss: 0.0459 - val_acc: 0.9881
Epoch 6/15
60000/60000 [==============================] - 2392s 40ms/step - loss: 0.0479 - acc: 0.9880 - val_loss: 0.0400 - val_acc: 0.9898
Epoch 7/15
60000/60000 [==============================] - 2392s 40ms/step - loss: 0.0279 - acc: 0.9917 - val_l

In [ ]:
#evaluate the model
#for more information run model.metrices_names == > ['loss' , 'acc']
scores = model_object.evaluate(x = x_test,y=y_test,batch_size = 32,verbose = 1)

print("The scores : " +scores)